In [41]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

### 1. Importing the Dataset

In [71]:
df = pd.read_csv('house_price_full+(2).csv')
df.head()

,bedrooms,sqft_living,price
0,3,1340,313000
1,5,3650,2384000
2,3,1930,342000
3,3,2000,420000
4,4,1940,550000


In [72]:
X = df.copy()
## Removing the target
y = X.pop('price')

In [73]:
X.head(2)

,bedrooms,sqft_living
0,3,1340
1,5,3650


In [74]:
y.head(2)

0     313000
1    2384000
Name: price, dtype: int64

In [75]:
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X),columns=X.columns)
X

,bedrooms,sqft_living
0,-0.433198,-0.753258
1,1.675735,1.457330
2,-0.433198,-0.188649
3,-0.433198,-0.121661
4,0.621269,-0.179079
...,...,...
494,0.621269,0.873582
495,1.675735,2.299459
496,-0.433198,-0.724549
497,-0.433198,-0.179079


In [76]:
## performing log transformation on the target variable
y = np.log(y)
y

0      12.653958
1      14.684290
2      12.742566
3      12.948010
4      13.217674
         ...    
494    13.380102
495    13.764217
496    12.128111
497    12.721886
498    12.254863
Name: price, Length: 499, dtype: float64

In [77]:
x1,x2 = X.loc[0]
print(x1,x2)

-0.43319764280264655 -0.7532575369937701


In [9]:
## bias
b = tf.Variable([0.1],dtype=tf.float32)
## weights
w1 = tf.Variable([0.2],dtype=tf.float32)
w2 = tf.Variable([0.15],dtype=tf.float32)

In [10]:
## Cumulative input
z = b + w1*x1 + w2*x2
h = tf.math.sigmoid(z)
print('The output of the first neuron is :',h)

The output of the first neuron is : tf.Tensor([0.47511354], shape=(1,), dtype=float32)


### Forward propagation with multiple tensors

In [14]:
## layer1 weights
##neuron 1
b1 = tf.Variable([0.1])
w11 = tf.Variable([0.2])
w12 = tf.Variable([0.15])
##neuron2
b2 = tf.Variable([0.25])
w21 = tf.Variable([0.5])
w22 = tf.Variable([0.6])

In [15]:
## forward pass
## neuron1
z1 = b1+w11*x1 + w12*x2
h1 = tf.math.sigmoid(z1)
print("The output from the first neuron is :",h1)

The output from the first neuron is : tf.Tensor([0.47511354], shape=(1,), dtype=float32)


In [16]:
## forward pass
## neuron 2
z2 = b2+w21*x1+w22*x2
h2 = tf.math.sigmoid(z2)
print("The output from the second neuron is:",h2)

The output from the second neuron is: tf.Tensor([0.39686295], shape=(1,), dtype=float32)


In [17]:
## Layer 2 weights
b1 = tf.Variable([0.4])
w11 = tf.Variable([0.3])
w12 = tf.Variable([0.2])

In [18]:
## forward pass
## second layer
z1 = b1+w11*h1+w12*h2
h1 = z1
print("The output from the first neuron is:",h1)

The output from the first neuron is: tf.Tensor([0.62190664], shape=(1,), dtype=float32)


In [22]:
## Computing the loss
y_true = y[0]
y_pred = h1.numpy()

In [23]:
## loss
L = 0.5*(y_true - y_pred)**2
print("The MSE error is:",L)

The MSE error is: [72.38514]


### Forward pass matrix multiplication

In [24]:
## Layer 1 weights
w1 = tf.Variable([[0.2,0.15],
                [0.5,0.6]],dtype=tf.float32)
## layer1 bias
b1 = tf.Variable([[0.1],
                 [0.25]],dtype=tf.float32)

In [25]:
## layer 2 weights
w2 = tf.Variable([[0.3,0.2]],dtype=tf.float32)
## bias
b2 = tf.Variable([0.4],dtype=tf.float32)

In [26]:
## data 
X = tf.constant([[x1,x2]],dtype=tf.float32)

In [28]:
## forward pass layer 1
z1 = tf.matmul(w1,tf.transpose(X))+b1
h1 = tf.math.sigmoid(z1)
print(h1)

tf.Tensor(
[[0.47511354]
 [0.39686295]], shape=(2, 1), dtype=float32)


In [29]:
## forward pass layer 2
z2 = tf.matmul(w2,h1)+b2
h2 = z2

In [31]:
y_pred = h2.numpy()
loss = 0.5*(y_true-y_pred)**2
print(loss)

[[72.38514]]


In [48]:
def random_init_params():
    w1 = tf.Variable(tf.random.uniform((2,2)))
    w2 = tf.Variable(tf.random.uniform((1,2)))
    b1 = tf.Variable(tf.random.uniform((2,1)))
    b2 = tf.Variable(tf.random.uniform((1,1)))
    return w1,b1,w2,b2

In [49]:
w1,b1,w2,b2 = random_init_params()

In [50]:
print("The initial 1st layer weight are:\n",w1.numpy())
print("-----------------------------------------------")
print("The initial 2nd layer weight are:\n",w2.numpy())
print("-----------------------------------------------")
print("The initial 1st layer bias are:\n",b1.numpy())
print("-----------------------------------------------")
print("The initial 2nd layer bias are:\n",b2.numpy())

The initial 1st layer weight are:
 [[0.6153872  0.79780936]
 [0.7474631  0.08413625]]
-----------------------------------------------
The initial 2nd layer weight are:
 [[0.13052309 0.4823085 ]]
-----------------------------------------------
The initial 1st layer bias are:
 [[0.78104067]
 [0.5509958 ]]
-----------------------------------------------
The initial 2nd layer bias are:
 [[0.3513968]]


In [51]:
def forward_prop(x,w1,b1,w2,b2):
    z1 = tf.matmul(w1,tf.transpose(x))+b1
    h1 = tf.math.sigmoid(z1)
    z2 = tf.matmul(w2,h1)+b2
    h2 = z2
    return h2

In [52]:
x = tf.constant([[x1,x2]],dtype=tf.float32)
y = y[0]

In [53]:
y_pred = forward_prop(x,w1,b1,w2,b2)

In [54]:
## Loss 
L = 0.5*(y-y_pred)**2
print("The MSE error is",L)

The MSE error is tf.Tensor([[71.751305]], shape=(1, 1), dtype=float32)


### Backpropagation

In [55]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

### Part 1: Autogradients

In [56]:
x = tf.Variable(0.0)  ## add gradient tape
lr = eta = 0.1

In [57]:
with tf.GradientTape() as tape:
    y = x**2+4*x
grad = tape.gradient(y,x)  ##dy/dx

In [59]:
grad.numpy()  ## dy/dx =2x+4, x=0, dy/dx=4

4.0

In [60]:
x.assign_sub(lr*grad)  ##x_neew = x_old -lr*dy/dx

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=-0.4>

In [61]:
x.numpy()

-0.4

### Part 2:Gradient Descent

In [62]:
x = tf.Variable(0.0)
lr = 0.1
for i in range(10):
    with tf.GradientTape() as tape:
        y = x**2+4*x
    grad = tape.gradient(y,x)
    x.assign_sub(lr*grad)
    print(x.numpy())

-0.4
-0.72
-0.9760001
-1.1808001
-1.34464
-1.4757121
-1.5805696
-1.6644557
-1.7315645
-1.7852516


## Backpropagation on the dataset

In [78]:
X

,bedrooms,sqft_living
0,-0.433198,-0.753258
1,1.675735,1.457330
2,-0.433198,-0.188649
3,-0.433198,-0.121661
4,0.621269,-0.179079
...,...,...
494,0.621269,0.873582
495,1.675735,2.299459
496,-0.433198,-0.724549
497,-0.433198,-0.179079


In [79]:
y

0      12.653958
1      14.684290
2      12.742566
3      12.948010
4      13.217674
         ...    
494    13.380102
495    13.764217
496    12.128111
497    12.721886
498    12.254863
Name: price, Length: 499, dtype: float64

In [80]:
x1,x2

(-0.43319764280264655, -0.7532575369937701)

In [84]:
df_scaled = X.copy()
df_scaled.head(2)

,bedrooms,sqft_living
0,-0.433198,-0.753258
1,1.675735,1.457330


In [83]:
df.head(2)

,bedrooms,sqft_living,price
0,3,1340,313000
1,5,3650,2384000


In [110]:
x = tf.constant([[x1,x2]],dtype=tf.float32)
Y = y[0]

In [99]:
## function to randomly initialize the weights and bias of the network
def random_init_params():
    w1 = tf.Variable(tf.random.uniform((2,2)))
    b1 = tf.Variable(tf.random.uniform((2,1)))
    w2 = tf.Variable(tf.random.uniform((1,2)))
    b2 = tf.Variable(tf.random.uniform((1,1)))
    return w1,b1,w2,b2

## function for forward propagation
def forward_prop(x,w1,b1,w2,b2):
    z1 = tf.matmul(w1,tf.transpose(x)) + b1
    h1 = tf.math.sigmoid(z1)
    z2 = tf.matmul(w2,h1)+b2
    h2 = z2
    return h2

In [100]:
w1,b1,w2,b2 = random_init_params()

In [101]:
with tf.GradientTape() as tape:
    y_pred= forward_prop(x,w1,b1,w2,b2)
    loss = 0.5*(Y - y_pred)**2

In [102]:
gw1 , gb1,gw2,gb2 = tape.gradient(loss,[w1,b1,w2,b2])

In [103]:
gw1

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[0.4803885 , 0.83531445],
       [0.8527377 , 1.4827669 ]], dtype=float32)>

In [104]:
gb1

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[-1.1089361],
       [-1.9684727]], dtype=float32)>

In [105]:
gw2

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[-7.2578382, -5.2738485]], dtype=float32)>

In [106]:
gb2

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-11.050636]], dtype=float32)>

In [107]:
lr = 0.01
w1.assign_sub(lr*gw1)
b1.assign_sub(lr*gb1)
w2.assign_sub(lr*gw2)
b2.assign_sub(lr*gb2)

<tf.Variable 'UnreadVariable' shape=(1, 1) dtype=float32, numpy=array([[1.0806936]], dtype=float32)>

### Part 4 : Training Loop

In [115]:
def train_one_step(x,y,w1,b1,w2,b2):
    y_true = y
    with tf.GradientTape() as g:
        y_pred = forward_prop(x,w1,b1,w2,b2)
        
        ## Loss
        loss= 0.5*(y_true - y_pred)**2
        
    ## Gradient calculation
    print("************************************************")
    print("GRADIENTS")
    print("************************************************")
    gw1 ,gb1,gw2,gb2 = g.gradient(loss,[w1,b1,w2,b2])
    print("the gradient of 1st layer weights are:\n",gw1.numpy())
    print("-------------------------------------------------")
    print("the gradient of 2nd layer weights are:\n",gw2.numpy())
    print("-------------------------------------------------")
    print("the gradient of 1st layer bias are:\n",gb1.numpy())
    print("-------------------------------------------------")
    print("the gradient of 2nd layer bias are:\n",gb2.numpy())
    print("-------------------------------------------------")
    print("The initial loss:",loss)
    
    #Gradient descent:
    lr = 0.2
    w1.assign_sub(lr*gw1)
    b1.assign_sub(lr*gb1)
    w2.assign_sub(lr*gw2)
    b2.assign_sub(lr*gb2)
    y_pred = forward_prop(x,w1,b1,w2,b2)
    loss= 0.5*(y_true - y_pred)**2
    print("************************************************")
    print("NEW UPDATES")
    print("************************************************")
    print("the updated 1st layer weights are\n:",w1.numpy())
    print("-------------------------------------------------")
    print("The updated 2nd layer weights are\n:",w2.numpy())
    print("-------------------------------------------------")
    print("The updated 1st layer bias are\n:",b1.numpy())
    print("-------------------------------------------------")
    print("The updated 2nd layer bias are\n:",b2.numpy())
    print("-------------------------------------------------")
    print("The updated loss is:",loss)
    return w1,b1,w2,b2,loss

In [116]:
w1,b1,w2,b2 = random_init_params()
w1,b1,w2,b2,loss = train_one_step(x,Y,w1,b1,w2,b2)

************************************************
GRADIENTS
************************************************
the gradient of 1st layer weights are:
 [[0.02431548 0.0422805 ]
 [0.21466026 0.3732579 ]]
-------------------------------------------------
the gradient of 2nd layer weights are:
 [[-7.6577587 -6.0803123]]
-------------------------------------------------
the gradient of 1st layer bias are:
 [[-0.05613021]
 [-0.49552497]]
-------------------------------------------------
the gradient of 2nd layer bias are:
 [[-12.007863]]
-------------------------------------------------
The initial loss: tf.Tensor([[72.09439]], shape=(1, 1), dtype=float32)
************************************************
NEW UPDATES
************************************************
the updated 1st layer weights are
: [[0.18614332 0.18174054]
 [0.54136896 0.6135037 ]]
-------------------------------------------------
The updated 2nd layer weights are
: [[1.5517848 1.381156 ]]
-------------------------------------

- Clearly there is a decrease in the loss we have seen.